## 6.1. 사례 : 전략 패턴의 리팩토링

### 6.1.1. 고전적인 전략

In [1]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

In [2]:
jack = Customer('jack', 100)

### 1) Context

In [3]:
class LineItem:
    
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
        
    def total(self):
        return self.price * self.quantity

In [4]:
class Order:
    
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
        
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

### 2) Strategy(전략)

In [5]:
class Promotion(ABC):
    
    @abstractmethod # 이 클래스를 상속하는 클래스는 반드시 이 메서드를 선언해야함
    def discount(self, order):
        pass

### 3) Concrete Strategy(구체적인 전략)

In [6]:
# 충성도 점수가 1000점 이상인 고객에게 전체 5% 할인 적용
class FidelityPromo(Promotion):
    
    def discount(self, order):
        return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

# 20개 이상의 동일 상품을 구입하면 10% 할인 적용
class BulkItemPromo(Promotion):
    
    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * 0.1
        return discount

# 10종류 이상의 상품을 구입하면 전체 7% 할인 적용
class LargeOrderPromo(Promotion):
    
    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * 0.07
        return 0

In [7]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, 5),
       LineItem('apple', 10, 1.5),
       LineItem('watermellon', 5, 5.0)]

In [8]:
Order(joe, cart, FidelityPromo())

<Order total: 60.00 due: 60.00>

In [9]:
Order(ann, cart, FidelityPromo())

<Order total: 60.00 due: 57.00>

------------------

### 6.1.2. 함수지향 전략

* Concrete Strategy(구체적인 전략)들은 속성은 없고 discount()라는 메서드 하나만을 가진 클래스이므로, 이를 함수로 대신할 수 있음.

In [10]:
class Order:
    
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
    
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount
    
    def __repr__(self):
        fmt ='<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

In [11]:
def fidelity_promo(order):
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

def bulk_item_promo(order):
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * 0.1
    return discount

def large_order_promo(order):
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * 0.07
    return 0

In [12]:
Order(joe, cart, fidelity_promo)

<Order total: 60.00 due: 60.00>

* 코드가 짧아졌으며 Order 객체를 사용하는 방법도 더 간단해짐.